## Семинар 8: "Современные модели для NLP"

ФИО: Сим Роман Дмитриевич

### На семинаре мы разберем [код трансфомера на pytorch](https://nlp.seas.harvard.edu/2018/04/03/attention.html)

###  ДЗ [3 балла]

Обратите внимание, что в этой работе вам потребуется скачать модель весом ~150MB, также ее вычисление занимает определенное время, так что рекомендуется считать эту задачу на [google colab](https://colab.research.google.com/).

In [1]:
import torch
!pip install --upgrade sentencepiece
!pip install --upgrade transformers
from transformers import *

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 1.3 MB 23.1 MB/s 
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 5.8 MB 30.0 MB/s 
     |████████████████████████████████| 182 kB 17.7 MB/s 
     |████████████████████████████████| 7.6 MB 60.9 MB/s 


/usr/local/lib/python3.8/dist-packages/transformers/generation_utils.py:24: FutureWarning: Importing `GenerationMixin` from `src/transformers/generation_utils.py` is deprecated and will be removed in Transformers v5. Import as `from transformers import GenerationMixin` instead.
  warnings.warn(
/usr/local/lib/python3.8/dist-packages/transformers/generation_tf_utils.py:24: FutureWarning: Importing `TFGenerationMixin` from `src/transformers/generation_tf_utils.py` is deprecated and will be removed in Transformers v5. Import as `from transformers import TFGenerationMixin` instead.
  warnings.warn(


In [2]:
MODEL = (MobileBertForMaskedLM, MobileBertTokenizer, 'google/mobilebert-uncased')

model_class, tokenizer_class, pretrained_weights = MODEL
# Load pretrained model/tokenizer
tokenizer = tokenizer_class.from_pretrained(pretrained_weights)
model = model_class.from_pretrained(pretrained_weights)

Downloading:   0%|          | 0.00/232k [00:00<?, ?B/s]

loading file vocab.txt from cache at /root/.cache/huggingface/hub/models--google--mobilebert-uncased/snapshots/1f90a6c24c7879273a291d34a849033eba2dbc0f/vocab.txt
loading file added_tokens.json from cache at None
loading file special_tokens_map.json from cache at None
loading file tokenizer_config.json from cache at None


Downloading:   0%|          | 0.00/847 [00:00<?, ?B/s]

loading configuration file config.json from cache at /root/.cache/huggingface/hub/models--google--mobilebert-uncased/snapshots/1f90a6c24c7879273a291d34a849033eba2dbc0f/config.json
Model config MobileBertConfig {
  "_name_or_path": "google/mobilebert-uncased",
  "architectures": [
    "MobileBertForPreTraining"
  ],
  "attention_probs_dropout_prob": 0.1,
  "classifier_activation": false,
  "classifier_dropout": null,
  "embedding_size": 128,
  "hidden_act": "relu",
  "hidden_dropout_prob": 0.0,
  "hidden_size": 512,
  "initializer_range": 0.02,
  "intermediate_size": 512,
  "intra_bottleneck_size": 128,
  "key_query_shared_bottleneck": true,
  "layer_norm_eps": 1e-12,
  "max_position_embeddings": 512,
  "model_type": "mobilebert",
  "normalization_type": "no_norm",
  "num_attention_heads": 4,
  "num_feedforward_networks": 4,
  "num_hidden_layers": 24,
  "pad_token_id": 0,
  "transformers_version": "4.25.1",
  "trigram_input": true,
  "true_hidden_size": 128,
  "type_vocab_size": 2,
  "u

Downloading:   0%|          | 0.00/147M [00:00<?, ?B/s]

loading weights file pytorch_model.bin from cache at /root/.cache/huggingface/hub/models--google--mobilebert-uncased/snapshots/1f90a6c24c7879273a291d34a849033eba2dbc0f/pytorch_model.bin
Some weights of the model checkpoint at google/mobilebert-uncased were not used when initializing MobileBertForMaskedLM: ['cls.seq_relationship.bias', 'cls.seq_relationship.weight']
- This IS expected if you are initializing MobileBertForMaskedLM from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing MobileBertForMaskedLM from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
All the weights of MobileBertForMaskedLM were initialized from the model checkpoint at google/mobilebert-uncased.
If your task is similar to the task the m

In [3]:
input_ids = tokenizer.encode("Here is some text to encode", add_special_tokens=True)  # Add special tokens takes care of adding [CLS], [SEP], <s>... tokens in the right way for each model.
print(input_ids)

[101, 2182, 2003, 2070, 3793, 2000, 4372, 16044, 102]


In [4]:
tokenizer.decode(input_ids)

'[CLS] here is some text to encode [SEP]'

In [5]:
input_ids[4] = tokenizer.mask_token_id
tokenizer.decode(input_ids)

'[CLS] here is some [MASK] to encode [SEP]'

In [6]:
input_batch = torch.tensor(input_ids).unsqueeze(0) # batch_size 1
with torch.no_grad():
    res = model(input_batch)[0]

In [7]:
prob = torch.nn.functional.softmax(res, dim=-1)
new_ids = prob.max(-1)[0]

In [8]:
tokenizer.decode(new_ids.numpy()[0, :].tolist())

'[PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD]'

In [9]:
GPT_TEXTS = [
    "In a shocking finding, scientist discovered a herd of unicorns living in a remote, previously unexplored valley, in the Andes Mountains. Even more surprising to the researchers was the fact that the unicorns spoke perfect English.",
    "A train carriage containing controlled nuclear materials was stolen in Cincinnati today. Its whereabouts are unknown."
    ]

Ваша задача - сгенерировать продолжение текстов, на которых демонстрировалась работа GPT-2 с помощью загруженной модели (DistillBERT). Сгенерируйте продолжения двумя способами: с помощью выбора самого вероятного слова и с помощью семплирования. Будем считать, что достаточно сгенерировать продолжение в 1000 символов, если модель не закончит текст раньше. Также можно попробовать сравнить эту генерацию с какой-нибудь легковесной gpt, например, "sshleifer/tiny-gpt2".

# BERT

In [10]:
from transformers import DistilBertTokenizer, DistilBertForMaskedLM

In [82]:
MODEL = (DistilBertForMaskedLM, DistilBertTokenizer, 'distilbert-base-uncased')

model_class, tokenizer_class, pretrained_weights = MODEL
# Load pretrained model/tokenizer
tokenizer = tokenizer_class.from_pretrained(pretrained_weights)
model = model_class.from_pretrained(pretrained_weights)

loading file vocab.txt from cache at /root/.cache/huggingface/hub/models--distilbert-base-uncased/snapshots/1c4513b2eedbda136f57676a34eea67aba266e5c/vocab.txt
loading file added_tokens.json from cache at None
loading file special_tokens_map.json from cache at None
loading file tokenizer_config.json from cache at /root/.cache/huggingface/hub/models--distilbert-base-uncased/snapshots/1c4513b2eedbda136f57676a34eea67aba266e5c/tokenizer_config.json
loading configuration file config.json from cache at /root/.cache/huggingface/hub/models--distilbert-base-uncased/snapshots/1c4513b2eedbda136f57676a34eea67aba266e5c/config.json
Model config DistilBertConfig {
  "_name_or_path": "distilbert-base-uncased",
  "activation": "gelu",
  "architectures": [
    "DistilBertForMaskedLM"
  ],
  "attention_dropout": 0.1,
  "dim": 768,
  "dropout": 0.1,
  "hidden_dim": 3072,
  "initializer_range": 0.02,
  "max_position_embeddings": 512,
  "model_type": "distilbert",
  "n_heads": 12,
  "n_layers": 6,
  "pad_tok

In [83]:
input_ids = tokenizer.encode(GPT_TEXTS[0], add_special_tokens=True)  # Add special tokens takes care of adding [CLS], [SEP], <s>... tokens in the right way for each model.
print(input_ids)

[101, 1999, 1037, 16880, 4531, 1010, 7155, 3603, 1037, 14906, 1997, 21830, 2015, 2542, 1999, 1037, 6556, 1010, 3130, 16655, 2595, 24759, 19574, 3028, 1010, 1999, 1996, 18096, 4020, 1012, 2130, 2062, 11341, 2000, 1996, 6950, 2001, 1996, 2755, 2008, 1996, 21830, 2015, 3764, 3819, 2394, 1012, 102]


In [84]:
tokenizer.decode(input_ids)

'[CLS] in a shocking finding, scientist discovered a herd of unicorns living in a remote, previously unexplored valley, in the andes mountains. even more surprising to the researchers was the fact that the unicorns spoke perfect english. [SEP]'

In [85]:
num_words = 100

In [86]:
print(tokenizer.eos_token)

Using eos_token, but it is not set yet.


None


## ArgMax 

In [87]:
input_ids = input_ids[1:-1]
tokenizer.decode(input_ids)

'in a shocking finding, scientist discovered a herd of unicorns living in a remote, previously unexplored valley, in the andes mountains. even more surprising to the researchers was the fact that the unicorns spoke perfect english.'

In [89]:
for i in range(num_words):
  input_ids.append(tokenizer.mask_token_id)
  input_batch = torch.tensor(input_ids).unsqueeze(0)
  with torch.no_grad():
    res = model(input_batch)[0]
  prob = torch.nn.functional.softmax(res, dim=-1)
  new_ids = prob.max(-1)[1]
  input_ids[-1] = new_ids[0].numpy()[-1]
  if len(input_ids) > i:
    break

In [90]:
tokenizer.decode(input_ids)

'in a shocking finding, scientist discovered a herd of unicorns living in a remote, previously unexplored valley, in the andes mountains. even more surprising to the researchers was the fact that the unicorns spoke perfect english. it also'

## Sampling

In [92]:
input_ids = tokenizer.encode(GPT_TEXTS[0], add_special_tokens=True)  # Add special tokens takes care of adding [CLS], [SEP], <s>... tokens in the right way for each model.
print(input_ids)

[101, 1999, 1037, 16880, 4531, 1010, 7155, 3603, 1037, 14906, 1997, 21830, 2015, 2542, 1999, 1037, 6556, 1010, 3130, 16655, 2595, 24759, 19574, 3028, 1010, 1999, 1996, 18096, 4020, 1012, 2130, 2062, 11341, 2000, 1996, 6950, 2001, 1996, 2755, 2008, 1996, 21830, 2015, 3764, 3819, 2394, 1012, 102]


In [93]:
tokenizer.decode(input_ids)

'[CLS] in a shocking finding, scientist discovered a herd of unicorns living in a remote, previously unexplored valley, in the andes mountains. even more surprising to the researchers was the fact that the unicorns spoke perfect english. [SEP]'

In [94]:
num_words = 100
top_k = 40

In [95]:
from numpy import random

In [ ]:
for i in range(num_words):
  input_ids.append(tokenizer.mask_token_id)
  input_batch = torch.tensor(input_ids).unsqueeze(0)
  with torch.no_grad():
    res = model(input_batch)[0]
  prob = torch.topk(res, top_k, dim=2)
  indices = prob.indices
  probs = prob.values
  probs = torch.nn.functional.softmax(probs, dim=2)
  new_id = random.choice(indices[0][-1].numpy(), p=probs[0][-1].numpy())
  input_ids[-1] = new_id

In [ ]:
tokenizer.decode(input_ids)

# GPT

In [ ]:
from transformers import AutoTokenizer, AutoModelForCausalLM

tokenizer_gpt2 = AutoTokenizer.from_pretrained("gpt2")

model_gpt2 = AutoModelForCausalLM.from_pretrained("gpt2", pad_token_id=tokenizer_gpt2.eos_token_id)

In [ ]:
def generate_text(inp, model, tokenizer):
  input_ids = tokenizer.encode(inp, add_special_tokens=True)
  input_batch = torch.tensor(input_ids).unsqueeze(0)
  beam_output = model.generate(input_batch, max_length=100, num_beams=5, no_repeat_ngram_size=2)
  output = tokenizer.decode(beam_output[0], skip_special_tokens=True, clean_up_tokenization_spaces=True)
  return output

In [ ]:
text = generate_text(GPT_TEXTS[0], model_gpt2, tokenizer_gpt2)

In [ ]:
text

In [ ]:
from transformers import AutoTokenizer, AutoModelForCausalLM

tokenizer_tiny_gpt = AutoTokenizer.from_pretrained("sshleifer/tiny-gpt2")

model_tiny_gpt = AutoModelForCausalLM.from_pretrained("sshleifer/tiny-gpt2", pad_token_id=tokenizer_tiny_gpt.eos_token_id)

In [ ]:
text = generate_text(GPT_TEXTS[0], model_tiny_gpt, tokenizer_tiny_gpt)

In [ ]:
text